In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import losses
from tensorflow.keras import layers

In [2]:
print(tf.__version__)

In [3]:
#from google.colab import drive
#rive.mount('/content/drive')

In [4]:
## mozda zatreba ako je problem u ekstenzijama datoteka
#import imghdr     
#for file in image_file_list:    
#image = cv2.imread(file)
#file_type = imghdr.what(file)  
#if file_type != 'jpeg':  
#    print(file +  " - invalid - " +  str(file_type))  
#    cv2.imwrite(file, image)

In [5]:
import pathlib
#data_dir = pathlib.Path("slike_namjestaja")
train_dir = pathlib.Path("../input/furniture-detector/img/train")
test_dir = pathlib.Path("../input/furniture-detector/img/val")

In [6]:
image_count = len(list(train_dir.glob('*/*.jpg')))
print(image_count)

In [7]:
chair = list(train_dir.glob('chair/*'))
# Get the jpg files then sort them
chair_sort = sorted(chair)
PIL.Image.open(str(chair_sort[2]))

In [8]:
batch_size = 32
img_height = 200
img_width = 200

In [9]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  #validation_split=0.05,
  #subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [10]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  #validation_split=0.05,
  #subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [11]:
class_names = train_ds.class_names
print(class_names)

In [12]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [13]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  
  break

In [14]:
from tensorflow.keras import layers

#normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [15]:
#normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
#image_batch, labels_batch = next(iter(normalized_ds))
#first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
#print(np.min(first_image), np.max(first_image))

In [16]:
## Ako treba moze se koristiti data ugmentation da se poveca broj slika za trening, ideja je da augmentation 
## rotira postojece slike i takve opet koristi za trening
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [17]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [18]:
num_classes = len(class_names)

## architecture of the model:
inputs = layers.Input(shape=(img_height, img_width, 3) )
inputs1 = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
## data_augmentation
h1 = layers.Conv2D(32, 3, kernel_regularizer=tf.keras.regularizers.L2(0.01))(inputs1)
d1 = layers.MaxPooling2D()(h1)
d1 = layers.BatchNormalization()(d1)
d1 = layers.ReLU()(d1)
d1 = layers.Conv2D(32, 3, kernel_regularizer=tf.keras.regularizers.L2(0.01))(d1)
d1 = layers.BatchNormalization()(d1)
d1 = layers.ReLU()(d1)
d1 = layers.Conv2D(64, 3, kernel_regularizer=tf.keras.regularizers.L2(0.01))(d1)
d1 = layers.MaxPooling2D()(d1)
d1 = layers.BatchNormalization()(d1)
d1 = layers.ReLU()(d1)
h2 = layers.Conv2D(64, 3, kernel_regularizer=tf.keras.regularizers.L2(0.01))(d1)
h2 = layers.MaxPooling2D()(h2)
h2 = layers.BatchNormalization()(h2)
h2 = layers.ReLU()(h2)
h3 = layers.Conv2D(128, 3, kernel_regularizer=tf.keras.regularizers.L2(0.01))(h2)
h3 = layers.MaxPooling2D()(h3)
h3 = layers.BatchNormalization()(h3)
h3 = layers.ReLU()(h3)
h4 = layers.Conv2D(128, 3, kernel_regularizer=tf.keras.regularizers.L2(0.01))(h3)
h4 = layers.MaxPooling2D()(h4)
h4 = layers.BatchNormalization()(h4)
h4 = layers.ReLU()(h4)
h4 = layers.Conv2D(256, 2, kernel_regularizer=tf.keras.regularizers.L2(0.01))(h4)
h4 = layers.BatchNormalization()(h4)
h4 = layers.ReLU()(h4)

h4 = layers.Flatten()(h4)
h4 = layers.Dropout(0.4)(h4)
h4 = layers.Dense(256, kernel_regularizer=tf.keras.regularizers.L2(0.1),
    activity_regularizer=tf.keras.regularizers.L2(0.1))(h4)

out = layers.Dense(num_classes, activation = 'softmax')(h4)
model = tf.keras.Model(inputs, out)
model.summary()

In [19]:
optim = tf.keras.optimizers.Adam(lr = 0.001)
model.compile(
  optimizer=optim,
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [20]:
def scheduler(epoch):
    if epoch < 20:
        return 0.001
    else:
        return 0.0001

from tensorflow.keras import callbacks
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [21]:
no_epoch = 50

history = model.fit(
  train_ds,  ## nedostaju labele
  #validation_split=0.1,
  shuffle = True,
  epochs=no_epoch,
  callbacks = [callback]
)

In [22]:
## testing model
prediction = model.predict(test_ds)
model.evaluate(test_ds)

In [23]:
#model.save('20200918_94posto_acc.h5')
## crtanje funkcije gubitka
#print(history.history.keys())
# summarize history for accuracy
x = range(no_epoch)
y = np.array(history.history['accuracy'])*100

plt.plot(x, y, 'b')
#plt.plot(history.history['val_loss'], 'r', label = 'validation_loss')
plt.title('Tocnost modela - trening')
plt.ylabel('točnost (%)')
plt.xlabel('epoha')
#plt.yscale('log')
plt.legend()
plt.show()

In [24]:
################# analiza
# potrazimo pogreno interpretirane slike od strane neuronske mreze

## batch je problem u analizi pa ponovno učitavamo slike, ali bez batcha (sve su unutar jednog batcha)
## poredak nije isti po pokretanju celije, pa cemo za svaku sliku raditi predict modela za svaku sliku i analizirati
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  #validation_split=0.05,
  #subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=423)

# varijable
br = 0                      # counter
br_no = 0
test_len = len(list(test_dir.glob('*/*.jpg')))
print(test_len)

batch_no = 0
pred =[]
i = 0

# inicijaliziramo matricu koja ce na mjestu (i, j) imati broj previđanja NN da je klasa i zapravo klasa j
# to znači da su na dijagonali broj ispravnih predviđanja, a avan dijagonale pogrešni
# no dobra stvar je da imamo točno za svaku klasu i s čime ju mreža miješa koliko puta
matrix = np.zeros((5, 5))  # confusion matrix

plt.figure(figsize=(10, 10))
for images, labels in test_ds:
        for i in range(test_len):
            image = tf.reshape(images[i], (1,img_height,img_width, 3))
            pred = model.predict(image)
            matrix[labels[i], np.argmax(pred)] = matrix[labels[i], np.argmax(pred)]+1
            ## crtamo prvih 6 pogreno interpretiranih slika            
            if np.argmax(pred) != labels[i]:
                if br+1 < 10:
                    ax = plt.subplot(3, 3, br+1)
                    plt.imshow(images[i].numpy().astype("uint8"))
                    plt.title(class_names[np.argmax(pred)])
                    plt.axis("off")
                br = br+1
            br_no = br_no+1


    ## zbog slucajnog odabira parametara u pocetku, moguce je s istom arhitekturom dobiti različit broj pogresnih primjera

In [25]:
################# analiza
# potrazimo pogreno interpretirane slike od strane neuronske mreze

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  #validation_split=0.05,
  #subset="validation",
  seed=29255,
  image_size=(img_height, img_width),
  batch_size=423)

# varijable
br = 0                      # counter
br_no = 0
test_len = len(list(test_dir.glob('*/*.jpg')))
print(test_len)

batch_no = 0
pred =[]
i = 0
## crtamo tocno detektirane slike:
matrix = np.zeros((5, 5))  # confusion matrix

plt.figure(figsize=(10, 10))
for images, labels in test_ds:
        for i in range(test_len):
            image = tf.reshape(images[i], (1,img_height,img_width, 3))
            pred = model.predict(image)
            matrix[labels[i], np.argmax(pred)] = matrix[labels[i], np.argmax(pred)]+1
            ## crtamo prvih 6 pogreno interpretiranih slika            
            if np.argmax(pred) == labels[i]:
                if br+1 < 10:
                    ax = plt.subplot(3, 3, br+1)
                    plt.imshow(images[i].numpy().astype("uint8"))
                    plt.title(class_names[np.argmax(pred)])
                    plt.axis("off")
                br = br+1
            br_no = br_no+1


    ## zbog slucajnog odabira parametara u pocetku, moguce je s istom arhitekturom dobiti različit broj pogresnih primjera

In [26]:
# analiza točnosti predviđanja neuronske mreže
# vidljivo da sada acccuracy je kao i gore sa model.evaluate()

true = np.trace(matrix)
false = np.sum(matrix) - true
print('Ukupni broj slika u testu:', br_no)
print("Broj tocno interpretiranih slika: ", true)
print("Broj pogresno interpretiranih slika: ", false)
print("Postotak pogresno interpretiranih slika: ", false / test_len*100, "%")
print('Tocnost modela:', true/test_len*100, '°%')
print(matrix)


In [27]:
#### analiza točnosti po klasama
## s obzirom da je najmanje stolova u datasetovima, tu očekujemo manju točnost, no je li tako?

suma = np.zeros(num_classes)
for i in range(num_classes):
    for j in range(num_classes):
        suma[i]= suma[i]+matrix[i, j]

print(suma)

print('Tocnost po klasama:')
for i in range(num_classes):
    print('Tocnost prepoznavanja klase  #', class_names[i], '#  iznosi', matrix[i, i]/suma[i]*100, '%')
    